## TODO Intro

#### Part 0: Setup

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, load_component, Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline
from promptflow import PFClient

In [5]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)
print(os.getenv("WORKSPACE_NAME"))
print(os.getenv("AZURE_OPENAI_ENDPOINT"))

victor-aml
https://victor-aoai.openai.azure.com/


In [9]:
import subprocess

api_key = os.getenv("AZURE_OPENAI_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
name = "aoai-connection"

command = f"pf connection create --file ../src/promptflow/connections/connect_aoai.yaml --set api_key={api_key} api_base={api_base} --name {name}"
result = subprocess.run(command,
                        shell=True,
                        check=True,
                        text=True,
                        capture_output=True
                        )

print(result.stdout)

{
    "name": "aoai-connection",
    "module": "promptflow.connections",
    "created_date": "2023-12-11T13:24:51.137301",
    "last_modified_date": "2024-03-13T13:59:44.268921",
    "type": "azure_open_ai",
    "api_key": "******",
    "api_base": "https://victor-aoai.openai.azure.com/",
    "api_type": "azure",
    "api_version": "2023-07-01-preview",
    "auth_mode": "key"
}



#### Part 1: Run Evaluation Flows (to Test)

In [ ]:
# PFClient can help manage your runs and connections.
pf = PFClient()

# Define Flows and Data
grey_customer_flow = "../src/promptflow/evaluation_flows/gpt_eval" # set flow directory
data = "../data/inputs/simple_chat_sample_inputs.csv" # set the data file

# Run evaluation flows  to evaluate chat results
grey_customer = pf.run(
    flow=grey_customer_flow,
    data=data,
    stream=False,
    column_mapping={  # map the url field from the data to the url input of the flow
      "question": "${data.input}",
      "response": "${run.outputs.output}",
    }
)

# TODO: Show outputs

In [ ]:
# TODO: Add other eval flows

#### Part 2: Load Flows as Components to be used in Pipeline Job

In [ ]:
# flow_component = load_component("../../flows/standard/web-classification/flow.dag.yaml")

#### Part 3: Create Pipeline and Deploy as Batch Endpoint

#### Part 4: Invoke Batch Endpoint